In [35]:
from aocd.models import Puzzle
import re

puzzle = Puzzle(year=2023, day=24)

def parses(input):
    def parse_line(line):
        nums = [int(i) for i in re.findall('-?\d+', line)]
        return (nums[:3], nums[3:])
    return [parse_line(line) for line in input.strip().split('\n')]

data = parses(puzzle.input_data)

In [36]:
sample = parses("""19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3""")

In [3]:
import itertools

In [7]:
def intersect(p1,v1,p2,v2):
    ##### MATH
    # x = ox + vx * t
    # y = oy + vy * t

    # (x-ox)/vx = (y-oy)/vy

    # x/vx - y/vy = ox/vx - oy/vy
    # x/vx2 - y/vy2 = ox/vx2 - oy/vy2
    A = np.array([
        [1/v1[0], -1/v1[1]],
        [1/v2[0], -1/v2[1]],
    ])
    b = np.array([
        [p1[0]/v1[0] - p1[1]/v1[1]],
        [p2[0]/v2[0] - p2[1]/v2[1]],
    ])
    if np.linalg.det(A) == 0:
        return None, None
    return np.linalg.solve(A, b)

In [8]:
# x = ox + vx * t
# t = (x-ox)/vx

In [10]:
def solve_a(data, low=200000000000000, high=400000000000000):

    intersections = 0
    for (p1, v1), (p2, v2) in itertools.combinations(data, 2):
        x, y = intersect(p1,v1,p2,v2)
        if x is None:
            continue
        t1 = (x-p1[0])/v1[0]
        t2 = (x-p2[0])/v2[0]
        if t1 >= 0 and t2 >= 0:
            if low<=x<=high and low <= y <= high:
                intersections += 1

    return intersections

In [11]:
solve_a(sample, 7, 27)

2

In [12]:
solve_a(data)

16050

In [ ]:
(ox - Ox) / (Vx - vx) =  t

In [ ]:
(ox - Ox) (Vy - vy)  =  (oy - Oy) (Vx - vx) 
ox Vy - ox vy - Ox Vy + Ox vy = oy Vx

In [ ]:
x = ox + vx * t
y = oy + vy * t
z = oz + vz * t

x = Ox + Vx * t
y = Oy + Vy * t
z = Oz + Vz * t

In [103]:
from z3 import Int, Solver

In [118]:
def solve_b(data):
    solver = Solver()
    Px, Py, Pz = Int('Px'), Int('Py'), Int('Pz')
    Vx, Vy, Vz = Int('Vx'), Int('Vy'), Int('Vz')
    for i, ((px,py,pz), (vx,vy,vz)) in enumerate(data):
        t = Int(f't{i}')
        solver.add(Px+Vx*t==px+vx*t)
        solver.add(Py+Vy*t==py+vy*t)
        solver.add(Pz+Vz*t==pz+vz*t)
        solver.add(t >= 0)
    assert str(solver.check()) == 'sat'
    m = solver.model()
#     print(m)
    val = (m[Px].as_long() + m[Py].as_long() + m[Pz].as_long())
    return val

In [119]:
solve_b(sample)

47

In [120]:
%%time
solve_b(data)

CPU times: user 2.43 s, sys: 7.49 ms, total: 2.43 s
Wall time: 2.46 s


669042940632377

In [116]:
%%time
solve_b(data, Real)

CPU times: user 179 ms, sys: 3.26 ms, total: 182 ms
Wall time: 183 ms


669042940632377

In [ ]:
P + V * t_i = P_i + V_i * t_i

(P-P_i) - t_i * (V-V_i) = 0

P-P_i (parallel-to) V-V_i

(P-P_i) X (V-V_i)

dpx, dpy, dpz <- P - P_i
dvx, dvy, dvz <- V - V_i

| i j k |
| dpx dpy dpz |
| dvx dvy dvz |



In [ ]:
(P-P_i) x (V-V_i) = (P-P_j) x (V-V_j)

(P x V) - (P x V_i) - (P_i x V) + (P_i x V_i) = (P x V) - (P x V_j) - (P_j x V) + (P_j x V_j)

- (P x V_i) - (P_i x V) + (P_i x V_i) = - (P x V_j) - (P_j x V) + (P_j x V_j)

SyntaxError: invalid syntax (2051995775.py, line 1)

# No parallel lines in input

In [61]:
import math

In [87]:
from collections import defaultdict
dups = defaultdict(list)
for pos, vel in data:
    vx, vy, vz = vel
    d = math.gcd(vx,vy,vz) * vx // abs(vx)
    v = (vx//d, vy//d, vz//d)
    dups[v].append(pos)

In [88]:
for vel, pos_list in dups.items():
    if len(pos_list) > 1:
        print(vel, pos_list)
        

In [89]:
from toolz import valmap
from collections import Counter

In [90]:
Counter(dict(valmap(len, dups)).values())

Counter({1: 300})

In [ ]:
# x = ox + vx * t
# x2 = ox2 + vx2 * t
# (ox - ox2)/(vx2 - vx) = t

In [ ]:
from fractions import Fraction

def intersect(p1,v1,p2,v2):
    if v2-v1 == 0:
        return None
    t = Fraction(p1-p2, -v1+v2)
    if t > 0:
        return t

In [27]:
data = sample
low, high = 7, 27

collisions = 0

for (p1, v1), (p2, v2) in itertools.combinations(data, 2):
    tx = intersect(p1[0], v1[0], p2[0], v2[0])
    ty = intersect(p1[1], v1[1], p2[1], v2[1])
    print(p1, p2, tx, ty)
    if tx is not None and ty is not None: # and tx == ty:
        x = p1[0] + v1[0] * tx
        y = p1[1] + v1[1] * ty
        if low <= x <= high and low <= y <= high:
            collisions += 1
            
collisions

[19, 13, 30] [18, 19, 22] 1 3
17 16
[19, 13, 30] [20, 25, 34] None 4
[19, 13, 30] [12, 31, 28] 7 6
5 19
[19, 13, 30] [20, 19, 15] None 1
[18, 19, 22] [20, 25, 34] 2 6
16 13
[18, 19, 22] [12, 31, 28] None 12
[18, 19, 22] [20, 19, 15] None None
[20, 25, 34] [12, 31, 28] 8 None
[20, 25, 34] [20, 19, 15] None None
[12, 31, 28] [20, 19, 15] None None


2

[([19, 13, 30], [-2, 1, -2]),
 ([18, 19, 22], [-1, -1, -2]),
 ([20, 25, 34], [-2, -2, -4]),
 ([12, 31, 28], [-1, -2, -1]),
 ([20, 19, 15], [1, -5, -3])]

In [15]:
def solve_a(data):
    pass

In [2]:
def solve_b(data):
    pass